In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [12]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 80

In [13]:
train_dir = '/content/drive/MyDrive/My_Research/VIPERC/dataset/train'
test_dir = '/content/drive/MyDrive/My_Research/VIPERC/dataset/test'

train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 5025 images belonging to 13 classes.
Found 1256 images belonging to 13 classes.


In [14]:
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

for layer in base_model.layers:
    layer.trainable = False

In [15]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [16]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min')

In [18]:
model.fit(train_generator, epochs=EPOCHS, validation_data=test_generator, callbacks=[checkpoint, early_stopping])

Epoch 1/80
158/158 [==============================] - 409s 3s/step - loss: 0.3064 - accuracy: 0.9224 - val_loss: 0.0896 - val_accuracy: 0.9697
Epoch 2/80
158/158 [==============================] - 27s 173ms/step - loss: 0.0399 - accuracy: 0.9893 - val_loss: 0.0301 - val_accuracy: 0.9904
Epoch 3/80
158/158 [==============================] - 27s 169ms/step - loss: 0.0243 - accuracy: 0.9938 - val_loss: 0.0593 - val_accuracy: 0.9841
Epoch 4/80
158/158 [==============================] - 26s 166ms/step - loss: 0.0144 - accuracy: 0.9962 - val_loss: 0.0340 - val_accuracy: 0.9912
Epoch 5/80
158/158 [==============================] - 27s 169ms/step - loss: 0.0070 - accuracy: 0.9982 - val_loss: 0.0296 - val_accuracy: 0.9912
Epoch 6/80
158/158 [==============================] - 26s 165ms/step - loss: 0.0113 - accuracy: 0.9976 - val_loss: 0.0410 - val_accuracy: 0.9904
Epoch 7/80
158/158 [==============================] - 26s 168ms/step - loss: 0.0100 - accuracy: 0.9978 - val_loss: 0.0335 - val_accu

In [20]:
!mv /content/best_model.keras /content/drive/MyDrive/My_Research/VIPERC/

In [21]:
from tensorflow.keras.models import load_model

best_model = load_model('/content/drive/MyDrive/My_Research/VIPERC/best_model.keras')

In [22]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

test_loss, test_accuracy = best_model.evaluate(test_generator)
print(f'\nTest Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

class_labels = list(test_generator.class_indices.keys())
predictions = best_model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Lấy nhãn thực tế từ generator
true_classes = test_generator.classes

# Tính các chỉ số đánh giá
accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes, average='weighted')
recall = recall_score(true_classes, predicted_classes, average='weighted')
f1 = f1_score(true_classes, predicted_classes, average='weighted')

# In ra kết quả
print('Accuracy:', accuracy * 100)
print('Precision:', precision * 100)
print('Recall:', recall * 100)
print('F1 Score:', f1* 100)

# In báo cáo chi tiết
print('\nClassification Report:')
print(classification_report(true_classes, predicted_classes, target_names=class_labels, digits=4))

# Tính accuracy cho từng nhãn
conf_matrix = confusion_matrix(true_classes, predicted_classes)
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

print('\nAccuracy for each class:')
for class_name, class_accuracy in zip(class_labels, class_accuracies):
    print(f'{class_name}: {class_accuracy:.2f}')

40/40 [==============================] - 6s 124ms/step - loss: 0.0216 - accuracy: 0.9952

Test Loss: 0.0216
Test Accuracy: 99.52%
40/40 [==============================] - 5s 124ms/step
Accuracy: 99.52229299363057
Precision: 99.52672734645596
Recall: 99.52229299363057
F1 Score: 99.52236598406238

Classification Report:
              precision    recall  f1-score   support

 cong_chieng     1.0000    1.0000    1.0000        60
     dan_bau     1.0000    1.0000    1.0000        79
      dan_co     1.0000    0.9787    0.9892        47
      dan_da     1.0000    0.9837    0.9918       123
     dan_day     0.9892    1.0000    0.9946        92
  dan_nguyet     0.9841    0.9920    0.9880       125
     dan_sen     0.9924    0.9924    0.9924       131
  dan_t_rung     0.9942    1.0000    0.9971       171
    dan_tinh     1.0000    1.0000    1.0000       124
   dan_tranh     1.0000    0.9939    0.9969       164
   dan_ty_ba     1.0000    1.0000    1.0000        48
        khen     1.0000    1.00